In [1]:
import sys
import torch as t
import numpy as np
from matplotlib import pyplot as plt
import json
from itertools import product
sys.path.append('/workspace/wilson/Finite-groups/src')
from model import MLP3, MLP4, InstancedModule
from utils import *
from group_data import *
from jaxtyping import Float
from typing import Union
from einops import repeat
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import copy
import math
from itertools import product
from llc import *
from sklearn.decomposition import PCA
%load_ext autoreload
%autoreload 2

device = t.device("cuda" if t.cuda.is_available() else "cpu")

In [2]:
def get_pc0(A, thresh=1e-6, verbose=True):
    '''
    Returns (max principal component pc1, A orthogonalized by pc1)
    '''
    x = t.randn(A.shape[1], 1)
    x_prev = t.zeros_like(x)
    x /= x.norm()
    while (x - x_prev).norm() > thresh:
        x_prev = x
        x = A.T @ (A @ x)
        x /= x.norm()
        if verbose:
            print((x - x_prev).norm())
    return x, A - (A @ x) * x.T

In [3]:
path = '/workspace/models/2024_08_05_18_31_53_S_5__times_A_5__Z_2__'
model, params = load_models(path, sel='final')

In [4]:
params.group_string = 'S(5)'

In [5]:
dataset = GroupData(params)

Intersection size: 14400/14400 (1.00)
Added 14400 elements from intersection
Added 0 elements from group 0: S(5)
Train set size: 14400/14400 (1.00)


In [6]:
loader = DataLoader(dataset=dataset, batch_size=150, shuffle=False)
X, _ = next(iter(loader))
X = X.to(device)

In [7]:
itr = load_models_itr(path)

In [ ]:
outputs = []
for model in tqdm(itr, total=1000):
    output = model.to(device)(X)
    outputs.append(output.detach().cpu())
    
outputs = t.stack(outputs)  # epoch batch instance N

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
outputs = einops.rearrange(outputs, 'epoch batch instance vocab -> (epoch instance) (batch vocab)')

In [ ]:
pca0, A0 = get_pc0(outputs.T)

In [ ]:
pca1, A1 = get_pc0(A0)

In [ ]:
pca2, A2 = get_pc0(A1)

In [ ]:
pcs = t.concat([pca0, pca1, pca2], dim=1)

In [ ]:
pcs = einops.rearrange(pcs, '(epoch instance) m -> epoch instance m', epoch=1000, instance=100)

In [ ]:
G0 = [5, 55, 90, 93]
G1 = [8, 44, 66, 92, 97, 99]

In [ ]:
pc_idx0 = 0
pc_idx1 = 1
for i in range(pcs.shape[1]):
    if i in G0:
        c = 'r'
        alpha = 1.
    elif i in G1:
        c = 'g'
        alpha = 1.
    else:
        c = 'b'
        alpha = 0.05
    plt.plot(pcs[100:,i,pc_idx0].numpy(), pcs[100:,i,pc_idx1].numpy(), c=c, alpha=alpha)
plt.show()

In [ ]:
pcs.shape

In [ ]:
i = 9
plt.plot(pcs[100:,i,0].numpy(), pcs[100:,i,1].numpy())
plt.show()